In [1]:
from utils.enums import Enum
from utils.bigquery_tools import buscar_producto_bd_solo_filtros

class TipoMecanica(Enum):
    GASOLINA = "GASOLINA"
    DIESEL = "DIESEL"

# Dict de filtros (solo lo que tu lógica acepta)
filtros_test = {
    'solo_electricos': 'no',
    'tipo_carroceria': ['SUV', 'PICKUP'],
    'tipo_mecanica': [TipoMecanica.GASOLINA, TipoMecanica.DIESEL]
}

# Dict de pesos (soft weights)
pesos_test = {
    'estetica': 0.03,
    'premium': 0.16,
    'singular': 0.16,
    'altura_libre_suelo': 0.06,
    'traccion': 0.32,
    'reductoras': 0.25
}


In [2]:
resultados = buscar_producto_bd_solo_filtros(filtros_test, pesos_test, k=10)
for coche in resultados:
    print("🚗", coche['marca'], coche['modelo'], coche['tipo_mecanica'], coche['tipo_carroceria'], "| Score:", round(coche['score_total'], 3))



🧠 SQL que se ejecuta:

    SELECT
      nombre,
      ID,
      marca,
      modelo,
      cambio_automatico,
      tipo_mecanica,
      tipo_carroceria,
      indice_altura_interior,
      batalla,
      estetica,
      premium,
      singular,
      altura_libre_suelo,
      traccion,
      reductoras,
      (
        estetica * @peso_estetica
        + premium * @peso_premium
        + singular * @peso_singular
        + indice_altura_interior/1000 * @peso_altura
        + batalla/1000 * @peso_batalla
        + CASE
            WHEN traccion = 'ALL' THEN 1.0
            WHEN traccion = 'RWD' THEN 0.5
            ELSE 0.0
          END * @peso_traccion
        + (CASE WHEN reductoras THEN 1 ELSE 0 END) * @peso_reductoras
      ) AS score_total
    FROM `web_cars.match_coches_pruebas`
    WHERE 1=1
      AND cambio_automatico = TRUE
    
      AND tipo_carroceria IN UNNEST(@tipo_carroceria)
    ORDER BY score_total DESC
    LIMIT @k

🚗 Bentley Bentayga GASOLINA SUV | Score: 6.625
🚗 Be